In [1]:
import csv
import gzip
import json
import os
import pandas as pd
import sys

In [2]:
DATA_DIR = "/home/waren/sourced/data/GH_archive/"
OUTPUT_DIR = "/home/waren/sourced/data/datasets"

In [3]:
# approx 500 GB on disk
#!wget http://data.gharchive.org/{2015..2018}-{01..12}-{01..31}-{0..23}.json.gz
!wget http://data.gharchive.org/2015-01-01-0.json.gz -P $DATA_DIR

--2019-03-25 15:46:22--  http://data.gharchive.org/2015-01-01-0.json.gz
Resolving data.gharchive.org (data.gharchive.org)... 104.27.165.156, 104.27.164.156, 2606:4700:30::681b:a59c, ...
Connecting to data.gharchive.org (data.gharchive.org)|104.27.165.156|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2744253 (2.6M) [application/gzip]
Saving to: ‘/home/waren/sourced/data/GH_archive/2015-01-01-0.json.gz.1’

2015-01-01-0.json.g 100%[===================>]   2.62M  8.37MB/s    in 0.3s    

2019-03-25 15:46:23 (8.37 MB/s) - ‘/home/waren/sourced/data/GH_archive/2015-01-01-0.json.gz.1’ saved [2744253/2744253]



In [4]:
COLUMNS = ["COMMENT_ID", "COMMIT_ID", "URL", "AUTHOR", "CREATED_AT", "BODY"]
PATH_TO_CSV = os.path.join(OUTPUT_DIR, "PR_comments.csv")
for root, d, files in os.walk(DATA_DIR):
    for i, f in enumerate(files):
        comments_list = []
        comments = pd.DataFrame(columns = COLUMNS)
        sys.stderr.write("%d\r" % i)
        path = os.path.join(root, f)
        if path.endswith(".gz"):
            try:
                for line in gzip.open(path).readlines():
                    event = json.loads(line)
                    if event["type"] == "PullRequestReviewCommentEvent":
                        comments_list.append([event["payload"]["comment"]["id"],
                                              event["payload"]["comment"]["commit_id"],
                                              event["payload"]["comment"]["html_url"],
                                              event["payload"]["comment"]["user"]["login"],
                                              event["payload"]["comment"]["created_at"],
                                              event["payload"]["comment"]["body"]])
            except EOFError:
                continue
        df = pd.DataFrame(comments_list, columns=COLUMNS)
        comments = comments.append(df, ignore_index=True)
        comments.to_csv(PATH_TO_CSV, mode="a", header=(not os.path.exists(PATH_TO_CSV)), index=False)

In [5]:
comments = pd.read_csv(PATH_TO_CSV)
comments.head()

,COMMENT_ID,COMMIT_ID,URL,AUTHOR,CREATED_AT,BODY
0,22396772,47c1cd384b3deb0427e7e546e8d6b4346a65d816,https://github.com/selfhub/selfhub/pull/124#di...,zindlerb,2015-01-01T00:00:08Z,:sparkles: :pray: :sparkles:
1,22396775,a12155068a4feb03bf6b149b60830154ff52db0f,https://github.com/tsuru/tsuru/pull/1044#discu...,msabramo,2015-01-01T00:00:53Z,> Can you remove this empty line?\r\n\r\nDone.
2,22396779,4c31e9b700de04c01f07c3f7d12afccbbf47237e,https://github.com/selfhub/selfhub/pull/125#di...,zindlerb,2015-01-01T00:01:44Z,"_.reduce would be more elegant, but your call"
3,22396780,2a0664c165aa3d1a4fec06d720d4ba393fefc7ae,https://github.com/emberjs/data/pull/2649#disc...,rwjblue,2015-01-01T00:01:54Z,Tiny nit-pick: can you name this something mea...
4,22396794,c431d49f3c1751d1c32ec1c122a7a1eeb06774a6,https://github.com/edx/edx-platform/pull/6399#...,bradenmacdonald,2015-01-01T00:02:33Z,"Yeah, I will just remove that. It is always True."
